In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.feature_extraction import text 
from sklearn.metrics import roc_auc_score
from collections import Counter
import re
import string


In [3]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('data/train.csv').fillna(' ')
test = pd.read_csv('data/test.csv').fillna(' ')

In [4]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])



In [5]:
word_vectorizer = TfidfVectorizer(min_df=11, stop_words=["the", 'to', 'of'])

In [6]:
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [7]:
classifier = LogisticRegression(C=2.4, tol=0.005, )

In [8]:
scores= []

for class_name in class_names:
    train_target = train[class_name]

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, scoring='roc_auc'))
    
    print('CV score for class {} is {}'.format(class_name, cv_score))
    scores.append(cv_score)

print('Total score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9723445688345457
CV score for class severe_toxic is 0.9839563792372176
CV score for class obscene is 0.9842562972283334
CV score for class threat is 0.9859281613031056
CV score for class insult is 0.9769681202633212
CV score for class identity_hate is 0.9739723195565144
Total score is 0.9795709744038397


In [9]:
all_text_list=all_text.apply(lambda x: x.lower().split())

In [10]:
all_text_list_of_words=[j for i in all_text_list for j in i]

In [11]:
sorted_list_of_words=Counter(all_text_list_of_words)

In [12]:
sorted_list_of_words.most_common(10)

[('the', 902050),
 ('to', 532403),
 ('of', 406605),
 ('a', 399980),
 ('and', 397026),
 ('i', 342269),
 ('you', 333262),
 ('is', 323691),
 ('that', 267385),
 ('in', 259696)]

In [13]:
submission = pd.DataFrame.from_dict({'id': test['id']})

In [14]:
for class_name in class_names:
    classifier.fit(train_word_features, train[class_name])
    submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

In [15]:
submission.to_csv('submission1.csv', index=False)